In [14]:

# Importing dependencies
import pandas as pd
from feast import FeatureStore
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage

# Getting our FeatureStore
store = FeatureStore(repo_path="breast_cancer/")

# Reading our targets as an entity DataFrame
entity_df = pd.read_parquet(path="breast_cancer/data/target_df.parquet")



### Retrieving features and creating a training dataset


In [15]:
# Getting the indicated historical features
# and joining them with our entity DataFrame

# it filters by entity id and date

training_data = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "df1_feature_view:mean radius",
        "df1_feature_view:mean texture",
        "df1_feature_view:mean perimeter",
        "df1_feature_view:mean area",
        "df1_feature_view:mean smoothness",
        "df2_feature_view:mean compactness",
        "df2_feature_view:mean concavity",
        "df2_feature_view:mean concave points",
        "df2_feature_view:mean symmetry",
        "df2_feature_view:mean fractal dimension",
        "df3_feature_view:radius error",
        "df3_feature_view:texture error",
        "df3_feature_view:perimeter error",
        "df3_feature_view:area error",
        "df3_feature_view:smoothness error",
        "df3_feature_view:compactness error",
        "df3_feature_view:concavity error",
        "df4_feature_view:concave points error",
        "df4_feature_view:symmetry error",
        "df4_feature_view:fractal dimension error",
        "df4_feature_view:worst radius",
        "df4_feature_view:worst texture",
        "df4_feature_view:worst perimeter",
        "df4_feature_view:worst area",
        "df4_feature_view:worst smoothness",
        "df4_feature_view:worst compactness",
        "df4_feature_view:worst concavity",
        "df4_feature_view:worst concave points",
        "df4_feature_view:worst symmetry",
        "df4_feature_view:worst fractal dimension"
    ]
)

In [16]:
training_data.to_df()

target                  event_timestamp  patient_id  mean radius  \
0         0 2020-11-10 11:25:48.285197+00:00           0        17.99   
1         0 2020-11-11 11:25:48.285197+00:00           1        20.57   
2         0 2020-11-12 11:25:48.285197+00:00           2        19.69   
3         0 2020-11-13 11:25:48.285197+00:00           3        11.42   
4         0 2020-11-14 11:25:48.285197+00:00           4        20.29   
..      ...                              ...         ...          ...   
564       0 2022-05-28 11:25:48.285197+00:00         564        21.56   
565       0 2022-05-29 11:25:48.285197+00:00         565        20.13   
566       0 2022-05-30 11:25:48.285197+00:00         566        16.60   
567       0 2022-05-31 11:25:48.285197+00:00         567        20.60   
568       1 2022-06-01 11:25:48.285197+00:00         568         7.76   

     mean texture  mean perimeter  mean area  mean smoothness  \
0           10.38          122.80     1001.0          0.11840   
1           17.77          132.90     1326.0          0.08474   
2           21.25          130.00     1203.0          0.10960   
3           20.38           77.58      386.1          0.14250   
4           14.34          135.10     1297.0          0.10030   
..            ...             ...        ...              ...   
564         22.39          142.00     1479.0          0.11100   
565         28.25          131.20     1261.0          0.09780   
566         28.08          108.30      858.1          0.08455   
567         29.33          140.10     1265.0          0.11780   
568         24.54           47.92      181.0          0.05263   

     mean compactness  mean concavity  ...  worst radius  worst texture  \
0             0.27760         0.30010  ...        25.380          17.33   
1             0.07864         0.08690  ...        24.990          23.41   
2             0.15990         0.19740  ...        23.570          25.53   
3             0.28390         0.24140  ...        14.910          26.50   
4             0.13280         0.19800  ...        22.540          16.67   
..                ...             ...  ...           ...            ...   
564           0.11590         0.24390  ...        25.450          26.40   
565           0.10340         0.14400  ...        23.690          38.25   
566           0.10230         0.09251  ...        18.980          34.12   
567           0.27700         0.35140  ...        25.740          39.42   
568           0.04362         0.00000  ...         9.456          30.37   

     worst perimeter  worst area  worst smoothness  worst compactness  \
0             184.60      2019.0           0.16220            0.66560   
1             158.80      1956.0           0.12380            0.18660   
2             152.50      1709.0           0.14440            0.42450   
3              98.87       567.7           0.20980            0.86630   
4             152.20      1575.0           0.13740            0.20500   
..               ...         ...               ...                ...   
564           166.10      2027.0           0.14100            0.21130   
565           155.00      1731.0           0.11660            0.19220   
566           126.70      1124.0           0.11390            0.30940   
567           184.60      1821.0           0.16500            0.86810   
568            59.16       268.6           0.08996            0.06444   

     worst concavity  worst concave points  worst symmetry  \
0             0.7119                0.2654          0.4601   
1             0.2416                0.1860          0.2750   
2             0.4504                0.2430          0.3613   
3             0.6869                0.2575          0.6638   
4             0.4000                0.1625          0.2364   
..               ...                   ...             ...   
564           0.4107                0.2216          0.2060   
565           0.3215                0.1628          0.2572   
566        

In [17]:
# Storing the dataset as a local file
dataset = store.create_saved_dataset(
    from_=training_data,
    name="breast_cancer_dataset",
    storage=SavedDatasetFileStorage("breast_cancer/data/breast_cancer_dataset.parquet")
)

/Users/taj/opt/anaconda3/lib/python3.9/site-packages/feast/feature_store.py:952: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


### Using the dataset to train a model


In [18]:
# Importing dependencies
from feast import FeatureStore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from joblib import dump

In [19]:
# Getting our FeatureStore
store = FeatureStore(repo_path="breast_cancer/")

In [20]:
# Retrieving the saved dataset and converting it to a DataFrame
training_df = store.get_saved_dataset(name="breast_cancer_dataset").to_df()

# Separating the features and labels
labels = training_df['target']
features = training_df.drop(
    labels=['target', 'event_timestamp', "patient_id"], 
    axis=1)

# Splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    stratify=labels)

/Users/taj/opt/anaconda3/lib/python3.9/site-packages/feast/feature_store.py:1005: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [21]:
# Creating and training LogisticRegression
reg = LogisticRegression()
reg.fit(X=X_train[sorted(X_train)], y=y_train)

# Saving the model
dump(value=reg, filename="model.joblib")

/Users/taj/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['model.joblib']

In [ ]:
# https://kedion.medium.com/creating-a-feature-store-with-feast-part-1-37c380223e2f

# more usages that filter by date exist here